In [10]:
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

def load_env():
    if load_dotenv():
        print("Environment variables loaded successfully.")
    else:
        print("Failed to load environment variables.")

In [11]:
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

In [12]:
import mysql.connector
from mysql.connector import Error

def connect_to_mysql(DB_USER,DB_HOST,DB_PORT,DB_PASSWORD,DB_NAME):
    try:
        connection = mysql.connector.connect(host = DB_HOST,
                             port = DB_PORT,
                             user = DB_USER,
                             password = DB_PASSWORD,
                             database = DB_NAME)
        if connection.is_connected():
            print("Connection réussie !")
            return connection
        else:
            print("Échec de la connexion.")
            return None
    except Error as e:
        print(f"Erreur: {e}")
        return None

In [13]:
connection = connect_to_mysql(DB_USER, DB_HOST, DB_PORT, DB_PASSWORD, DB_NAME)

Connection réussie !


In [14]:
import pandas as pd
def fetch_data_from_sql(query, connection):
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

In [15]:
query = "SELECT * FROM airline_delay limit 100"

In [16]:
df = fetch_data_from_sql(query,connection=connection)

In [17]:
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2020,12,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",44,3,1.63,0,...,0,1.25,0,1,89,56,0,3,0,30
1,2020,12,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",90,1,0.96,0,...,0,0.00,0,0,23,22,0,1,0,0
2,2020,12,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",88,8,5.75,0,...,0,0.65,0,1,338,265,0,45,0,28
3,2020,12,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",184,9,4.17,0,...,0,3.00,0,0,508,192,0,92,0,224
4,2020,12,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",76,11,4.78,0,...,0,1.00,1,0,692,398,0,178,0,116


In [18]:
import pandas as pd

def dataframe_to_json(df):
    json_data = df.to_json(orient='records')
    return json_data

In [19]:
json_data = dataframe_to_json(df.head(100))

In [20]:
json_data

'[{"year":2020,"month":12,"carrier":"9E","carrier_name":"Endeavor Air Inc.","airport":"ABE","airport_name":"Allentown\\/Bethlehem\\/Easton, PA: Lehigh Valley International","arr_flights":44,"arr_del15":3,"carrier_ct":1.63,"weather_ct":0,"nas_ct":0.12,"security_ct":0,"late_aircraft_ct":1.25,"arr_cancelled":0,"arr_diverted":1,"arr_delay":89,"carrier_delay":56,"weather_delay":0,"nas_delay":3,"security_delay":0,"late_aircraft_delay":30},{"year":2020,"month":12,"carrier":"9E","carrier_name":"Endeavor Air Inc.","airport":"ABY","airport_name":"Albany, GA: Southwest Georgia Regional","arr_flights":90,"arr_del15":1,"carrier_ct":0.96,"weather_ct":0,"nas_ct":0.04,"security_ct":0,"late_aircraft_ct":0.0,"arr_cancelled":0,"arr_diverted":0,"arr_delay":23,"carrier_delay":22,"weather_delay":0,"nas_delay":1,"security_delay":0,"late_aircraft_delay":0},{"year":2020,"month":12,"carrier":"9E","carrier_name":"Endeavor Air Inc.","airport":"AEX","airport_name":"Alexandria, LA: Alexandria International","arr_fl

In [21]:
import os

def get_api_key():
    load_dotenv()
    
    api_key = os.getenv("GROQ_API_KEY")
    
    return api_key

In [22]:
api_key= get_api_key()

In [23]:
model_name = "llama-3.1-8b-instant" 

In [24]:
from groq import Groq

def query_data(json_data, question, model_name, api_key):
    
    client = Groq(api_key=api_key)
    prompt = f"Based on the following relational data, answer the question:\n{json_data}\nQuestion: {question}"
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        model=model_name,
    )
    response = chat_completion.choices[0].message.content
    return response

In [25]:
question = "Summarize the data"

In [29]:
response = query_data(json_data=json_data,
                      question=question,
                      model_name=model_name,
                      api_key=api_key)

In [30]:
print(response)

The data provided represents a set of flight arrival records for passengers traveling on Endeavor Air Inc. within the United States during December 2020. The data captures flight information including arrival flights, on-time arrival records, and various factors that may contribute to flight delays such as carrier issues, weather, nationality and assimilation service (NAS) issues, and security checks.

Key observations can be made from the data:

1. **Total Arrivals**: The total number of flight arrivals for the month of December 2020 is 25,414. 

2. **Cities with most Flights**: The cities with the most flights during this period are Atlanta, Minneapolis, and Detroit for the airports ATL, MSP, and DTW respectively.

3. **Arrival Delays**: The average arrival delay for Endeavor Air Inc. flights in December 2020 is around 365 minutes. This implies that flights experienced an average delay of approximately 6 hours.

4. **Carrier vs. Weather vs. NAS Issues Delay minutes**: The average min

In [31]:
from fpdf import FPDF
import uuid
import os

def generate_pdf(response, file_path):
    """Generate a PDF file with the given response text."""
    try:
        # Create a PDF object
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()

        # Title: Change color and font style
        pdf.set_font("Arial", 'B', 16)  # Bold font with size 16 for the title
        pdf.set_text_color(0, 102, 204)  # Blue color for the title text
        pdf.cell(200, 10, txt="Answer to the Question", ln=True, align="C")  # Centered title

        pdf.ln(8)  # Add space after the title

        # Response Body: Change color and font style
        pdf.set_font("Arial", size=12)  # Normal font for the body
        pdf.set_text_color(0, 0, 0)  # Black color for the text

        # Titles and corresponding body text
        titles_and_responses = [
            ("Average Delay:", "394.56 minutes (approximately 6.57 hours)"),
            ("Response Body:", response),  # This can be any text passed as a response
            ("Next Title:", "Here is another section of content that follows the previous section.")
        ]

        for title, content in titles_and_responses:
            # Set the title in bold
            pdf.set_font("Arial", 'B', 12)  # Set bold font for the title
            pdf.multi_cell(0, 10, title)  # Print the title in bold

            # Set the content in normal font
            pdf.set_font("Arial", '', 12)  # Set normal font for the content
            pdf.multi_cell(0, 10, content)  # Print the content of the response

            pdf.ln(5)  # Add some space between sections

        # Add a footer with different text
        pdf.ln(10)  # Add space for the footer
        pdf.set_font("Arial", 'I', 10)  # Italic font for the footer
        pdf.set_text_color(128, 128, 128)  # Gray color for footer text
        pdf.cell(200, 10, txt="This document was generated automatically.", ln=True, align="C")

        # Save the file to the given path
        pdf.output(file_path)
        return file_path  # Return the path where the PDF was saved

    except Exception as e:
        print(f"Error generating PDF: {e}")
        return None  # Return None in case of an error

def generate_pdf_and_save(answer):
    """Create a PDF and save it in the 'responses' folder with sequential filenames."""
    try:
        # Ensure the 'responses' directory exists
        if not os.path.exists("responses"):
            os.makedirs("responses", exist_ok=True)  # Avoid error if folder already exists

        # Get the list of existing PDF files in the 'responses' directory
        existing_files = [f for f in os.listdir("responses") if f.startswith("response_") and f.endswith(".pdf")]

        # Generate the next sequential number for the filename
        next_index = len(existing_files) + 1
        unique_filename = f"responses/response_{next_index:02d}.pdf"  # Formats as response_01.pdf, response_02.pdf, etc.

        # Generate and save the PDF
        generated_pdf_path = generate_pdf(answer, unique_filename)

        if generated_pdf_path:
            return generated_pdf_path  # Return the path of the saved PDF
        else:
            print("Failed to generate PDF.")
            return None  # In case of failure

    except Exception as e:
        print(f"Error saving PDF: {e}")
        return None  # Return None if there was an error in the process


In [32]:
saved_pdf_path = generate_pdf_and_save(response)

if saved_pdf_path:
    print(f"Le PDF a été sauvegardé avec succès à : {saved_pdf_path}")
else:
    print("Une erreur est survenue lors de la génération du PDF.")

Le PDF a été sauvegardé avec succès à : responses/response_01.pdf
